In [53]:
import torch
from torchvision import transforms
from PIL import Image
import torchvision
from dataset import ImagesWithSaliency
from torchvision.io import read_image
import matplotlib.pyplot as plt
import numpy as np

In [47]:
fix_transform = transforms.Compose([
    transforms.Resize((128,128))
])
hm_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.Lambda(lambda x: x/255)
])

In [146]:
a = torch.Tensor([1,2,1])
print(a**2)

tensor([1., 4., 1.])


In [48]:
dataset = ImagesWithSaliency("./SalChartQA/train/raw_img/", "./SalChartQA/train/saliency_all/fix_maps/", "./SalChartQA/train/saliency_all/heatmaps/", fix_transform, hm_transform)

In [97]:
normalize = transforms.Normalize(0, 1)
def nss_npy(gt_locs, predicted_map):
    assert gt_locs.shape == predicted_map.shape, \
    'dim missmatch in nss_npy: %s vs %s' % (gt_locs.shape, predicted_map.shape)
    # predicted_map_norm = normalize(predicted_map)
    predicted_map_norm = (predicted_map - torch.mean(predicted_map))/torch.std(predicted_map)
    dot = predicted_map_norm * gt_locs
    N = gt_locs.sum()
    ret = torch.sum(dot)/N
    return ret

In [ ]:
def nss(fix, hm):
    return 

In [77]:
fix = dataset[200][2]

/home/v/mambaforge/envs/saliency/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [141]:
fix = dataset[0][2]
hm = dataset[200][3]

/home/v/mambaforge/envs/saliency/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [142]:
normalize = transforms.Normalize(hm.mean(), hm.std())

In [143]:
n = normalize.forward(hm)

In [145]:
n.mean()

tensor(3.8708e-09)

In [144]:
n.std()

tensor(1.)

In [78]:
torch.sum(normalize(dataset[200][3])*fix)/fix.sum()

tensor(0.3434)

In [76]:
nss_npy(fix, dataset[200][3])

tensor(2.4777)

In [36]:
from transformers import AutoTokenizer, BertModel, AutoImageProcessor, SwinModel, ViTModel

In [149]:
image_processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
model = SwinModel.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [154]:
image_processor = AutoImageProcessor.from_pretrained("microsoft/swin-base-patch4-window12-384")
model = SwinModel.from_pretrained("microsoft/swin-base-patch4-window12-384")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [148]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

86878584

In [155]:
image = torch.rand((224,224,3))
inputs = image_processor(image, return_tensors="pt")

In [156]:
outputs = model(**inputs, output_hidden_states=True)

In [157]:
outputs['last_hidden_state'].shape

torch.Size([1, 144, 1024])

In [31]:
outputs['hidden_states'][2].shape

torch.Size([1, 196, 512])

In [37]:
model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

In [45]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello, my dog dong dog dong ds dsa ds is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

In [48]:
model.config.hidden_size

768

In [8]:
type(model)

transformers.models.vit.modeling_vit.ViTModel

In [64]:
query = torch.rand((1, 13,768))
key = torch.rand((1, 197,768))
value = key

In [69]:
up = torch.nn.Upsample(11, mode='linear')

In [70]:
up(query.permute(0,2,1)).shape

torch.Size([1, 768, 11])

In [38]:
multihead_attn = torch.nn.MultiheadAttention(768, 12, batch_first=True)
attn_output, attn_output_weights = multihead_attn(query, key, value)

In [39]:
attn_output.shape

torch.Size([1, 13, 768])

In [1]:
from transformers import AutoImageProcessor, ViTMAEModel

/home/v/mambaforge/envs/saliency/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
image_processor = AutoImageProcessor.from_pretrained("facebook/vit-mae-large")
model = ViTMAEModel.from_pretrained("facebook/vit-mae-large")

In [14]:
img = torch.rand((1, 3, 224, 224))

In [15]:
inputs = image_processor(images=img, return_tensors="pt")
outputs = model(**inputs)

In [16]:
outputs['last_hidden_state'].shape

torch.Size([1, 50, 1024])

In [13]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")